In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, accuracy_score
import numpy as np

# Load Dataset 1 (predicting depth and time based on temperature and density)
df1 = pd.read_csv('dataset1.csv')

# Separate features and target variables for Dataset 1
X1 = df1[['Skin Temperature (°C)', 'Ointment Density (g/cm³)']]
y1 = df1[['Predicted Penetration Depth (mm)', 'Time to Reach Depth (min)']]

# Split data for training and testing for Dataset 1
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=42)

# Train a linear regression model for Dataset 1
lr_model = LinearRegression()
lr_model.fit(X1_train, y1_train)

# Function to predict depth and time based on temperature and density
def predict_depth_time(temp, density):
    prediction = lr_model.predict([[temp, density]])
    depth, time = prediction[0]
    return round(depth, 1), round(time, 1)

# Load Dataset 2 (predicting bloodstream mixing and skin reaction)
df2 = pd.read_csv('generated_skin_reaction_dataset.csv')

# Encode categorical variables in Dataset 2
le_skin_type = LabelEncoder()
df2['skin_type'] = le_skin_type.fit_transform(df2['skin_type'])  # e.g., dry=0, oily=1

le_bloodstream_mixing = LabelEncoder()
df2['bloodstream_mixing'] = le_bloodstream_mixing.fit_transform(df2['bloodstream_mixing'])  # No=0, Yes=1

le_skin_reaction = LabelEncoder()
df2['skin_reaction'] = le_skin_reaction.fit_transform(df2['skin_reaction'])  # no reaction=0, irritation=1, redness=2, etc.

# Separate features and target variables for Dataset 2
X2 = df2[['depth', 'time', 'skin_type', 'age']]
y_bloodstream = df2['bloodstream_mixing']
y_reaction = df2['skin_reaction']

# Split data for training and testing for Dataset 2
X2_train, X2_test, y_bloodstream_train, y_bloodstream_test = train_test_split(X2, y_bloodstream, test_size=0.2, random_state=42)
X2_train, X2_test, y_reaction_train, y_reaction_test = train_test_split(X2, y_reaction, test_size=0.2, random_state=42)

# Train RandomForest models for bloodstream mixing and skin reaction
rf_bloodstream = RandomForestClassifier()
rf_bloodstream.fit(X2_train, y_bloodstream_train)

rf_reaction = RandomForestClassifier()
rf_reaction.fit(X2_train, y_reaction_train)

# Function to predict bloodstream mixing and skin reaction with readable outputs
def predict_reaction_and_mixing(depth, time, skin_type, age):
    # Encode the skin type input before prediction
    skin_type_encoded = le_skin_type.transform([skin_type])[0]  # Convert 'dry' or 'oily' to corresponding integer
    input_data = [[depth, time, skin_type_encoded, age]]
    
    # Predict bloodstream mixing
    mixing_pred = rf_bloodstream.predict(input_data)[0]
    mixing_result = le_bloodstream_mixing.inverse_transform([mixing_pred])[0]  # Convert 0/1 back to 'No'/'Yes'
    
    # Predict skin reaction
    reaction_pred = rf_reaction.predict(input_data)[0]
    reaction_result = le_skin_reaction.inverse_transform([reaction_pred])[0]  # Convert to readable reaction label
    
    return mixing_result, reaction_result

# Example usage of the full prediction pipeline
temp_input = float(input("Enter the temperature: ")) # example temperature
density_input = float(input("Enter the density: "))  # example density
age_input = int(input("Enter the age: "))  # example age
skin_type_input = input("Enter the skin type: ")  # example skin type

# Step 1: Predict depth and time using Dataset 1 model
predicted_depth, predicted_time = predict_depth_time(temp_input, density_input)
print("Predicted Depth: ", predicted_depth, "Predicted Time: ", predicted_time)

# Step 2: Predict bloodstream mixing and skin reaction using Dataset 2 model
predicted_mixing, predicted_reaction = predict_reaction_and_mixing(predicted_depth, predicted_time, skin_type_input, age_input)
print("Predicted Bloodstream Mixing: ", predicted_mixing, "Predicted Skin Reaction: " ,predicted_reaction)


Enter the temperature:  35
Enter the density:  1.0658
Enter the age:  62
Enter the skin type:  oily


Predicted Depth:  5.6 Predicted Time:  3.6
Predicted Bloodstream Mixing:  Yes Predicted Skin Reaction:  mild irritation


C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [ ]:
df1

In [ ]:
df2